## 1. Abstract
- Brief summary
- Objectives:
  - Crawl web pages, extract content.
  - Build an index for search.
  - Enable ranked query results via Flask API.
- Next steps:
  - Improvements 

## 2. Overview
- Solution Outline: Explain the full workflow:
- Relevant Literature: Briefly cite research/articles on web crawling, TF-IDF, cosine similarity, or search systems.
- Proposed System: Highlight architecture and its intended functionality.

## 3. Design
- System Capabilities:
  - Max pages/depth crawling
  - TF-IDF index construction
  - Top-K query results
- Interactions: How crawler, indexer, and query processor communicate.
- Integration: File formats (JSON index, CSV queries), data flow.

## 4. Architecture
- Software Components:
  - Scrapy spider
  - Scikit-Learn indexer
  - Flask query processor
- Interfaces: APIs, endpoints, command-line interfaces.
- Implementation Notes: Python version, libraries (Scrapy, sklearn, Flask)

## 5. Operation
- Installation Instructions: Dependencies, environment setup, pip/venv usage.
- Software Commands:
  - Running crawler
  - Building the index
  - Running Flask server and querying
- Inputs: Seed URL, CSV queries, configuration options.

## 6. Conclusion
- Success/Failure:
- Outputs:

## 7. Data Sources
- Links to websites crawled (if allowed).

## 8. Test Cases
- Framework: Pytest, unittest, or custom scripts.
- Harness: How to test crawler, indexer, query processor.
- Coverage: Edge cases like empty pages, invalid URLs, long queries, or spelling errors.

## 9. Source Code
- https://docs.scrapy.org/en/latest/intro/tutorial.html

## 10. Bibliography

In [1]:
import nest_asyncio
nest_asyncio.apply()

import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.linkextractors import LinkExtractor
import os

### Step 1: Web Crawling

In [ ]:
os.makedirs("pages", exist_ok=True)

class NotebookCrawler(scrapy.Spider):
    name = "notebook_crawler"
    
    def __init__(self, seed_url, allowed_domain, max_pages, max_depth, *args, **kwargs):
        super(NotebookCrawler, self).__init__(*args, **kwargs)
        self.start_urls = [seed_url]
        self.allowed_domains = [allowed_domain]
        self.max_pages = max_pages
        self.visited = set()
        
        # Set max depth in custom settings
        self.custom_settings = {
            'DEPTH_LIMIT': max_depth,
            'AUTOTHROTTLE_ENABLED': True,
            'LOG_ENABLED': True,
            'CLOSESPIDER_PAGECOUNT': max_pages
        }
    
    def parse(self, response):
        # Stop if max pages reached
        if len(self.visited) >= self.max_pages:
            self.logger.info(f"Reached max pages limit: {self.max_pages}")
            return
        
        # Save HTML content
        url_safe = response.url.replace("https://", "").replace("http://", "").replace("/", "_")
        with open(f"pages/{url_safe}.html", "w", encoding="utf-8") as f:
            f.write(response.text)
        
        self.visited.add(response.url)
        self.logger.info(f"Saved page {len(self.visited)}/{self.max_pages}: {response.url}")
        
        # Extract and follow links
        if len(self.visited) < self.max_pages:
            links = LinkExtractor(allow_domains=self.allowed_domains).extract_links(response)
            for link in links:
                if link.url not in self.visited:
                    yield response.follow(link.url, self.parse)

# Usage with configurable parameters
process = CrawlerProcess(settings={
    'LOG_LEVEL': 'INFO',
})

process.crawl(
    NotebookCrawler,
    seed_url='https://quotes.toscrape.com',      
    allowed_domain='quotes.toscrape.com',      
    max_pages=100,                                  
    max_depth=5                                   
)
try:
    process.start()
except:
    pass

### Step 2: Indexing